# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 18, 2019**

In [1]:
using DataFrames
using BenchmarkTools

## Советы по производительности

### Доступ по номеру столбца быстрее, чем по имени

In [2]:
x = DataFrame(rand(5, 1000))
@btime $x[500];
@btime $x.x500;

  1.028 ns (0 allocations: 0 bytes)
  10.795 ns (0 allocations: 0 bytes)


### При работе с данными `DataFrame` используйте барьерные функции или аннотации типов

In [3]:
using Random
function f_bad() # будет медленной
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y, z = x[1], x[2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# запустите @code_warntype f_bad() и увидите
# что тип столбцов датафрейма неизвестен, что и породит много ненужных нативных команд


  84.426 ms (5999022 allocations: 122.06 MiB)


In [4]:
# Решение 1 заключается в использовании барьерной функции (ее можно использовать практически в любом коде)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # работать с пользовательской вложенной функцией
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    f_inner(x[1], x[2])
end

using LinearAlgebra
function f_inbuilt() # или используйте встроенную функцию, если это возможно
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    dot(x[1], x[2])
end

@btime f_barrier();
@btime f_inbuilt();

  12.337 ms (44 allocations: 30.52 MiB)
  12.230 ms (44 allocations: 30.52 MiB)


In [5]:
# Решение 2 заключается в предоставлении типов извлеченных столбцов - это проще, 
# но есть случаи, когда вы не будете знать эти типы. В этом примере предполагается, 
# что у вас есть мастер DataFrames как минимум с 31 августа 2018 г.
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y::Vector{Float64}, z::Vector{Float64} = x[1], x[2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  12.093 ms (44 allocations: 30.52 MiB)


### Возможность использования отложенного метода создания DataFrame

In [6]:
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f1()
    x = DataFrame(Float64, 10^4, 100) # мы работаем с DataFrame напрямую
    for c in 1:ncol(x)
        d = x[c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x) 
    # мы задерживаем создание DataFrame после того, как наша работа выполнена
end

@btime f1();
@btime f2();

  20.220 ms (1950037 allocations: 37.42 MiB)
  1.583 ms (936 allocations: 7.69 MiB)


### Вы можете добавить строки в «DataFrame» на месте, и это быстро

In [7]:
x = DataFrame(rand(10^6, 5))
y = DataFrame(transpose(1.0:5.0))
z = [1.0:5.0;]

@btime vcat($x, $y); # создает новый DataFrame - медленно
@btime append!($x, $y); # совершает действия над уже существующими - это быстро

x = DataFrame(rand(10^6, 5)) # сброс к той же начальной точке
@btime push!($x, $z); # добавить одну строку - самый быстрый вариант

  13.643 ms (134 allocations: 38.15 MiB)
  173.088 ns (1 allocation: 16 bytes)
  177.141 ns (5 allocations: 80 bytes)


### Допущение `missing`ов и `categorical`ов замедляет вычисления

In [8]:
using StatsBase

function test(data) # использует функцию countmap для проверки производительности
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.616 ms (8 allocations: 7.63 MiB)
 categorical:
  17.618 ms (4 allocations: 608 bytes)
String
 raw:
  22.131 ms (4 allocations: 608 bytes)
 categorical:
  28.753 ms (4 allocations: 608 bytes)
Union{Missing, Int64}
 raw:
  10.414 ms (4 allocations: 624 bytes)
 categorical:
  18.447 ms (4 allocations: 608 bytes)
Union{Missing, String}
 raw:
  24.373 ms (4 allocations: 608 bytes)
 categorical:
  38.403 ms (4 allocations: 608 bytes)


### При агрегировании используйте селектор столбцов и категориальные группировки переменных

In [9]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [10]:
@btime by($df, :x, v -> sum(v.y))

  315.468 ms (185 allocations: 433.19 MiB)


4×2 DataFrame
│ Row │ x    │ x1      │
│     │ Char │ Int64   │
├─────┼──────┼─────────┤
│ 1   │ 'b'  │ 2500383 │
│ 2   │ 'd'  │ 2498883 │
│ 3   │ 'c'  │ 2499690 │
│ 4   │ 'a'  │ 2501044 │

In [11]:
@btime by($df, :x, :y=>sum)

  189.816 ms (150 allocations: 356.89 MiB)


4×2 DataFrame
│ Row │ x    │ y_sum   │
│     │ Char │ Int64   │
├─────┼──────┼─────────┤
│ 1   │ 'b'  │ 2500383 │
│ 2   │ 'd'  │ 2498883 │
│ 3   │ 'c'  │ 2499690 │
│ 4   │ 'a'  │ 2501044 │

In [12]:
categorical!(df, :x);

In [13]:
@btime by($df, :x, :y=>sum)

  96.604 ms (169 allocations: 152.60 MiB)


4×2 DataFrame
│ Row │ x            │ y_sum   │
│     │ Categorical… │ Int64   │
├─────┼──────────────┼─────────┤
│ 1   │ 'a'          │ 2501044 │
│ 2   │ 'b'          │ 2500383 │
│ 3   │ 'c'          │ 2499690 │
│ 4   │ 'd'          │ 2498883 │

### Используйте представления вместо материализации нового DataFrame

In [14]:
x = DataFrame(rand(100, 1000))

100×1000 DataFrame. Omitted printing of 995 columns
│ Row │ x1        │ x2        │ x3        │ x4        │ x5         │
│     │ Float64   │ Float64   │ Float64   │ Float64   │ Float64    │
├─────┼───────────┼───────────┼───────────┼───────────┼────────────┤
│ 1   │ 0.793224  │ 0.312067  │ 0.455189  │ 0.966398  │ 0.507039   │
│ 2   │ 0.638377  │ 0.481852  │ 0.746923  │ 0.532772  │ 0.733642   │
│ 3   │ 0.393042  │ 0.641862  │ 0.628638  │ 0.0920168 │ 0.278093   │
│ 4   │ 0.905664  │ 0.40352   │ 0.239291  │ 0.234605  │ 0.502715   │
│ 5   │ 0.0988872 │ 0.566616  │ 0.684227  │ 0.92483   │ 0.688673   │
│ 6   │ 0.660386  │ 0.788396  │ 0.569571  │ 0.675332  │ 0.414782   │
│ 7   │ 0.644808  │ 0.486372  │ 0.235642  │ 0.58887   │ 0.199415   │
│ 8   │ 0.115018  │ 0.91365   │ 0.822832  │ 0.701608  │ 0.5834     │
│ 9   │ 0.0761012 │ 0.534577  │ 0.885232  │ 0.408539  │ 0.319737   │
│ 10  │ 0.35801   │ 0.993355  │ 0.9615    │ 0.242276  │ 0.591585   │
⋮
│ 90  │ 0.131346  │ 0.403369  │ 0.82626   │ 0.774454  │ 0.00376229 │
│ 91  │ 0.673136  │ 0.839864  │ 0.922507  │ 0.658339  │ 0.388018   │
│ 92  │ 0.0760549 │ 0.613235  │ 0.0966471 │ 0.155214  │ 0.755228   │
│ 93  │ 0.134479  │ 0.293058  │ 0.780751  │ 0.0758015 │ 0.988839   │
│ 94  │ 0.445355  │ 0.537575  │ 0.958733  │ 0.471407  │ 0.876185   │
│ 95  │ 0.568502  │ 0.313619  │ 0.526183  │ 0.324358  │ 0.408218   │
│ 96  │ 0.0124322 │ 0.472413  │ 0.0348449 │ 0.741943  │ 0.979908   │
│ 97  │ 0.195325  │ 0.0790011 │ 0.205489  │ 0.58371   │ 0.506079   │
│ 98  │ 0.567807  │ 0.968278  │ 0.512024  │ 0.416535  │ 0.216222   │
│ 99  │ 0.191299  │ 0.694953  │ 0.806579  │ 0.896626  │ 0.96852    │
│ 100 │ 0.0543214 │ 0.383794  │ 0.224767  │ 0.168838  │ 0.827307   │

In [15]:
@btime $x[1:1, :]

  138.281 μs (2511 allocations: 225.66 KiB)


1×1000 DataFrame. Omitted printing of 994 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │

In [16]:
@btime $x[1, :]

  21.590 ns (1 allocation: 32 bytes)


DataFrameRow. Omitted printing of 994 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │

In [21]:
@btime view($x, 1:1, :)

  22.104 ns (1 allocation: 48 bytes)


1×1000 SubDataFrame. Omitted printing of 994 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │

In [18]:
@btime $x[1:1, 1:20]

  4.553 μs (74 allocations: 7.84 KiB)


1×20 DataFrame. Omitted printing of 14 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │

In [19]:
@btime $x[1, 1:20]

  25.702 ns (2 allocations: 80 bytes)


DataFrameRow. Omitted printing of 14 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │

In [22]:
@btime view($x, 1:1, 1:20)

  25.703 ns (2 allocations: 96 bytes)


1×20 SubDataFrame. Omitted printing of 14 columns
│ Row │ x1       │ x2       │ x3       │ x4       │ x5       │ x6       │
│     │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ 0.793224 │ 0.312067 │ 0.455189 │ 0.966398 │ 0.507039 │ 0.385867 │